<a href="https://colab.research.google.com/github/Janaki74/Fmmllab24/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

01.(ANS)

When adjusting the percentage of the validation set:

1. **Increase Validation Set**:
   - **More Data for Validation**: Gives a better estimate of model performance.
   - **Reduced Training Data**: Can slightly decrease model accuracy due to less training data.
   
2. **Decrease Validation Set**:
   - **More Data for Training**: Can improve model accuracy by providing more data for learning.
   - **Less Reliable Validation**: May give less accurate estimates of performance due to smaller validation samples.

The balance between training and validation data affects both model learning and the reliability of performance estimates.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Split data with different validation sizes
train_sizes = [0.9, 0.8, 0.7]  # Percentages of data used for training
accuracies = []

for train_size in train_sizes:
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=train_size, random_state=42)

    # Train model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Validate model
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append((train_size, accuracy))

# Output results
for train_size, accuracy in accuracies:
    print(f"Train size: {train_size*100}%, Validation Accuracy: {accuracy*100:.2f}%")

Train size: 90.0%, Validation Accuracy: 100.00%
Train size: 80.0%, Validation Accuracy: 100.00%
Train size: 70.0%, Validation Accuracy: 100.00%


02.(ANS)

1. **Large Training, Small Validation**:
   - **High Training Accuracy**.
   - **Less Reliable Validation Accuracy**.
   - **Test Accuracy Prediction** may be inaccurate.

2. **Small Training, Large Validation**:
   - **Lower Training Accuracy** (risk of underfitting).
   - **More Reliable Validation Accuracy**.
   - **Test Accuracy Prediction** may be better, but the model may underperform.

3. **Balanced Split**:
   - **Good Training and Validation Accuracy**.
   - **Test Accuracy Prediction** is generally more reliable.

In [ ]:

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Generate synthetic classification data
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, random_state=42)

# Split the data into train, validation, and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Different training-validation splits
validation_sizes = [0.1, 0.2, 0.3, 0.4, 0.5]

results = []

for val_size in validation_sizes:
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=val_size, random_state=42)

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Calculate validation accuracy
    val_accuracy = accuracy_score(y_val, model.predict(X_val))

    # Calculate test accuracy
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    results.append((len(X_train), len(X_val), val_accuracy, test_accuracy))

# Print results
for train_size, val_size, val_acc, test_acc in results:
    print(f"Training size: {train_size}, Validation size: {val_size}, Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}")

Training size: 7200, Validation size: 800, Validation Accuracy: 0.8825, Test Accuracy: 0.8965
Training size: 6400, Validation size: 1600, Validation Accuracy: 0.8900, Test Accuracy: 0.8960
Training size: 5600, Validation size: 2400, Validation Accuracy: 0.8842, Test Accuracy: 0.8935
Training size: 4800, Validation size: 3200, Validation Accuracy: 0.8825, Test Accuracy: 0.8945
Training size: 4000, Validation size: 4000, Validation Accuracy: 0.8872, Test Accuracy: 0.8940


03.(ANS)

To achieve a balanced approach between training and validation:

1. **20-25% Validation**:
   - **Reliability**: Offers a solid estimate of model performance with a sufficiently large validation set.
   - **Training Data**: Keeps 75-80% for training, which is typically enough to build a strong model.

2. **30% Validation**:
   - **Larger Datasets**: Works well when you have a large dataset, allowing for a more detailed validation without significantly impacting the training process.
   - **Validation Accuracy**: Provides a more stable and reliable accuracy estimate.

### Factors to Consider:
- **Dataset Size**: Larger datasets can afford a higher percentage for validation.
- **Model Complexity**: More complex models may require more training data, favoring a smaller validation set.
- **Cross-Validation**: If uncertain, use cross-validation (e.g., 5-fold) to use the entire dataset for both training and validation across multiple iterations, providing a robust performance estimate.

In [ ]:

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Generate synthetic classification data
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, random_state=42)

# Split the data into a train+validation set and a test set (holdout test set)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the validation set percentages to experiment with
validation_percentages = [0.1, 0.15, 0.2, 0.25, 0.3]

results = []

for val_percentage in validation_percentages:
    # Split the training+validation set into actual training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=val_percentage, random_state=42)

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Calculate validation accuracy
    val_accuracy = accuracy_score(y_val, model.predict(X_val))

    # Calculate test accuracy
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    # Calculate the absolute difference between validation accuracy and test accuracy
    diff = abs(val_accuracy - test_accuracy)

    # Store the results
    results.append((val_percentage, len(X_train), len(X_val), val_accuracy, test_accuracy, diff))

# Print the results
print(f"{'Validation %':<12} {'Train Size':<12} {'Val Size':<10} {'Val Accuracy':<15} {'Test Accuracy':<15} {'Difference':<12}")
for val_percentage, train_size, val_size, val_acc, test_acc, diff in results:
    print(f"{val_percentage:<12.2f} {train_size:<12} {val_size:<10} {val_acc:<15.4f} {test_acc:<15.4f} {diff:<12.4f}")

# Find the validation percentage with the minimum difference
best_result = min(results, key=lambda x: x[-1])
best_val_percentage = best_result[0]
print(f"\nBest validation percentage: {best_val_percentage * 100:.2f}%")

Validation % Train Size   Val Size   Val Accuracy    Test Accuracy   Difference  
0.10         7200         800        0.8825          0.8965          0.0140      
0.15         6800         1200       0.8842          0.8950          0.0108      
0.20         6400         1600       0.8900          0.8960          0.0060      
0.25         6000         2000       0.8925          0.8950          0.0025      
0.30         5600         2400       0.8842          0.8935          0.0093      

Best validation percentage: 25.00%


> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


01.(ANS)

Averaging validation accuracy across multiple splits improves consistency by:

1. **Reducing Variance**: It mitigates the effect of any single split that might be unusually good or bad, leading to more stable results.
  
2. **Better Generalization**: It helps ensure that the model's performance is consistent across different subsets of the data, giving a better estimate of how it will perform on unseen data.

3. **Addressing Data Imbalance**: If some classes are underrepresented, multiple splits can ensure that each class gets appropriately represented in both training and validation sets.

4. **Robustness**: This approach makes the evaluation less sensitive to outliers or specific data distributions in a single split.

Overall, it leads to a more reliable assessment of the model's performance, especially in cases where data is limited or varied.

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Initialize model
model = RandomForestClassifier()

# Perform k-fold cross-validation
scores = cross_val_score(model, X, y, cv=5)  # cv=5 means 5-fold cross-validation

# Calculate average accuracy
average_accuracy = scores.mean()

print(f"Validation accuracies for each fold: {scores}")
print(f"Average validation accuracy: {average_accuracy:.4f}")

Validation accuracies for each fold: [0.96666667 0.96666667 0.93333333 0.96666667 1.        ]
Average validation accuracy: 0.9667


02.(ANS)

Averaging validation accuracy across multiple splits provides a more accurate estimate of test accuracy by:

1. **Reducing Overfitting Risk**: It prevents the model from overfitting to a specific train-validation split.
  
2. **Capturing Data Variability**: It accounts for the natural variability in the dataset, leading to a more representative assessment of model performance.

3. **Mitigating Randomness**: Reduces the influence of random factors that might skew results in a single split.

4. **Ensuring Generalization**: It better reflects how the model will perform on truly unseen data, as the evaluation considers multiple possible scenarios.

Overall, this approach gives a more robust and accurate prediction of test accuracy, enhancing confidence in the model's generalization ability.

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Calculate the mean and standard deviation of validation accuracy
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

# Output results
print(f"Mean Validation Accuracy: {mean_cv_score*100:.2f}%")
print(f"Validation Accuracy Std Dev: {std_cv_score*100:.2f}%")

Mean Validation Accuracy: 96.67%
Validation Accuracy Std Dev: 2.11%


03.(ANS)

Yes, the number of iterations typically has a significant effect on the accuracy of an estimate, especially in iterative algorithms or methods like Monte Carlo simulations, optimization algorithms, and machine learning models.

### Effects of Increasing Iterations:
1. **Accuracy Improvement**: More iterations generally lead to a more accurate estimate. As the number of iterations increases, the algorithm has more opportunities to refine its results, reducing the variance in the estimate.

2. **Convergence**: Many algorithms converge to a stable solution as the number of iterations increases. This means that after a certain point, additional iterations will result in diminishing improvements to the estimate.

3. **Reduction of Randomness**: In stochastic methods like Monte Carlo simulations, increasing the number of iterations reduces the impact of randomness, leading to a more reliable and stable estimate.

4. **Computational Cost**: However, increasing iterations also means more computational time and resources. It's essential to find a balance between the number of iterations and the desired level of accuracy, especially in resource-constrained environments.

### Diminishing Returns:
- **Law of Diminishing Returns**: Beyond a certain point, increasing the number of iterations may yield only marginal improvements in the estimate, making it important to assess when further iterations are no longer cost-effective.

In summary, higher iterations generally lead to better estimates, but it's important to balance this with the computational cost and the specific needs of the task at hand.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Sample DataFrame
data = pd.DataFrame({
    'feature1': [5, 3, 8, 1, 7],
    'feature2': [2, 4, 6, 8, 1],
    'label': [0, 1, 0, 1, 0]
})

# Sorting data by a feature
data_sorted = data.sort_values(by='feature1')

# Separating features and labels
X = data_sorted[['feature1', 'feature2']]
y = data_sorted['label']

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initializing and training a model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Making predictions and evaluating the model
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)

Accuracy: 0.5


04.(ANS)

When dealing with very small training or validation datasets, simply increasing the number of iterations in your machine learning model may not be sufficient to address the challenges. Here’s a concise overview of how increasing iterations interacts with small datasets:

### Challenges with Small Datasets:
1. **Overfitting**: With small datasets, models are more prone to overfitting, where they learn noise instead of patterns. Increasing iterations can exacerbate this issue as the model might fit the limited data too closely.

2. **Insufficient Training Data**: Even with many iterations, the model might not generalize well because it lacks enough data to learn effectively.

3. **High Variance**: Small datasets can lead to high variance in model performance, making the results unstable.

### Strategies Beyond Increasing Iterations:
1. **Data Augmentation**: Generate more training data by creating synthetic samples or using techniques like SMOTE (Synthetic Minority Over-sampling Technique).

2. **Cross-Validation**: Use techniques like k-fold cross-validation to make the most of your limited data and assess model performance more reliably.

3. **Regularization**: Apply regularization techniques to prevent overfitting, such as L1/L2 regularization, dropout in neural networks, or pruning in decision trees.

4. **Model Selection**: Choose simpler models that are less likely to overfit, or use ensemble methods to improve stability and performance.

5. **Transfer Learning**: Utilize pre-trained models and fine-tune them on your small dataset, leveraging knowledge from larger datasets.

In summary, while increasing iterations might help improve model performance to some extent, it’s crucial to address the limitations of small datasets through other strategies to achieve reliable and generalizable results.

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# Sample small dataset
data = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5],
    'feature2': [5, 4, 3, 2, 1],
    'label': [0, 1, 0, 1, 0]
})

# Features and labels
X = data[['feature1', 'feature2']]
y = data['label']

# Data scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Data augmentation (oversampling the minority class)
X_resampled, y_resampled = resample(X_scaled, y, replace=True, n_samples=10, random_state=42)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Initialize and train a model with regularization
model = LogisticRegression(penalty='l2')
model.fit(X_train, y_train)

# Making predictions
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

# Cross-validation
cv_scores = cross_val_score(model, X_resampled, y_resampled, cv=3)

print("Accuracy:", accuracy)
print("Cross-validation scores:", cv_scores)
print("Mean CV Score:", np.mean(cv_scores))

Accuracy: 0.6666666666666666
Cross-validation scores: [0.75       0.66666667 1.        ]
Mean CV Score: 0.8055555555555555


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.